In [1]:
import csv
import os
import glob
import pandas as pd
import nltk
import string
import re
from nltk import ngrams
import multiprocessing
import time

In [27]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [28]:
def find_ngrams(sentence, n_list):
    """Magic n-gram function."""
    inp, grams = sentence.split(), []
    for n in n_list:
      grams += [' '.join(x) for x in zip(*[inp[i:] for i in range(n)])]
    return grams

In [29]:
# Change to Dir of file
abspath = os.path.abspath(__file__)
dname = os.path.dirname(abspath)
os.chdir(dname)
#os.chdir("C://Users//Ruben//Documents//Scriptie//test-environment")
list_csv = glob.glob("*.csv")
list_csv

['nrmc-1844-1914.csv']

In [32]:
# Function for extracting ngrams and generating tokenized files

def PreProcess(filename):
    # Iterate over CSVs
    df = pd.read_csv(filename, sep = "\t")
    print(filename + " imported")
    
    # Drop duplicates
    lbf = len(df)
    df = df.drop_duplicates(subset=['id'], keep='first')
    laf = len(df)
    print(str(lbf-laf) + " duplicates dropped")
    
    # Check av. number of articles per year
    start_year = int(df.date[0][0:4])
    end_year = int(df.date[len(df)-1][0:4])
    
    try:
        aay = len(df) / (end_year - start_year) 
        if aay < 5000:
            df.date = df.date.str.slice(0, 4)
        if aay > 4999:
            df.date = df.date.str.replace("/", "")
            df.date = df.date.str.slice(0, 6)
            print('average number of articles/year = ', str(aay))

    except ZeroDivisionError:
        df.date = df.date.str.slice(0, 4)
  
        
        
    # Iterate over years
    for year in sorted(list(set(df.date))):
        df_subset_year = df[df.date == year]
        df_subset_year = df_subset_year.reset_index(drop=True)
        print(str(year) + " = " + str(len(df_subset_year)) + " articles")
        bigram_df = pd.DataFrame()
        unigram_df = pd.DataFrame()
        
        list_tok_articles_year = list()
        
        # Iterate over Articles in Subsetted CSV
        for i in range(0,(len(df_subset_year.ocr) - 1)):
            
            
            article = df_subset_year.ocr[i]
            
            tokens = clean_and_split_str(article)
            list_tok_articles_year.append(tokens)
            
            
            # Get ngrams from tokenized_article
            list_ngrams = find_ngrams(tokens, [1,2])
            list_unigrams = [word for word in list_ngrams if len(word.split(" ")) == 1]
            list_bigrams = [word for word in list_ngrams if len(word.split(" ")) == 2]

        
            df_unigrams_article = pd.DataFrame(list_unigrams)
            df_unigrams_article['year'] = year
            df_unigrams_article['count'] = 1
            df_unigrams_article.columns = ['ngram', 'year', 'count']
            unigram_df = unigram_df.append(df_unigrams_article)
            
            
            df_bigrams_article = pd.DataFrame(list_bigrams)
            df_bigrams_article['year'] = year
            df_bigrams_article['count'] = 1
            df_bigrams_article.columns = ['ngram', 'year', 'count']
            bigram_df = bigram_df.append(df_bigrams_article)
            print(str(i) + "/" + str(len(df_subset_year)) + ": " + df_subset_year.date[i] + " " + df_subset_year.id[i] + " processed")
        
        #Write list of senttok articles to one file
        txt_name = filename[0:4] + "_lines_" + str(year) + ".txt"
                        
        with open(txt_name, 'w') as f:
            for item in list_tok_articles_year:
                f.write("%s\n" % item)

        
        ## Group Ngram DFS
        unigram_df = unigram_df.groupby(['ngram', 'year']).sum()
        unigram_df = unigram_df.reset_index()
        unigram_df = unigram_df[unigram_df['count'] > 1]
        fn = filename[0:4] + "_unigram_" + str(year) + ".csv"
        unigram_df.to_csv(fn, index = False)
    
        bigram_df = bigram_df.groupby(['ngram', 'year']).sum()
        bigram_df = bigram_df.reset_index()
        bigram_df = bigram_df[bigram_df['count'] > 1]
        fn = filename[0:4] + "_" + "_bigram_" + str(year) + ".csv"
        bigram_df.to_csv(fn, index = False)

In [33]:
PreProcess(list_csv[0])

nrmc-1844-1914.csv imported
0 duplicates dropped
average number of articles/year =  6851.171428571429
184401 = 265 articles
0/265: 184401 ddd:010082921:mpeg21:a0001 processed
1/265: 184401 ddd:010082921:mpeg21:a0002 processed
2/265: 184401 ddd:010082921:mpeg21:a0003 processed
3/265: 184401 ddd:010082921:mpeg21:a0004 processed
4/265: 184401 ddd:010082921:mpeg21:a0005 processed
5/265: 184401 ddd:010082921:mpeg21:a0006 processed
6/265: 184401 ddd:010082921:mpeg21:a0007 processed
7/265: 184401 ddd:010082921:mpeg21:a0008 processed
8/265: 184401 ddd:010082921:mpeg21:a0009 processed
9/265: 184401 ddd:010082921:mpeg21:a0010 processed
10/265: 184401 ddd:010082921:mpeg21:a0011 processed
11/265: 184401 ddd:010082921:mpeg21:a0012 processed
12/265: 184401 ddd:010082921:mpeg21:a0022 processed
13/265: 184401 ddd:010082921:mpeg21:a0031 processed
14/265: 184401 ddd:010082921:mpeg21:a0034 processed
15/265: 184401 ddd:010082921:mpeg21:a0035 processed
16/265: 184401 ddd:010082921:mpeg21:a0036 processed
17

163/265: 184401 ddd:010082929:mpeg21:a0009 processed
164/265: 184401 ddd:010082929:mpeg21:a0010 processed
165/265: 184401 ddd:010082929:mpeg21:a0011 processed
166/265: 184401 ddd:010082929:mpeg21:a0012 processed
167/265: 184401 ddd:010082929:mpeg21:a0013 processed
168/265: 184401 ddd:010082929:mpeg21:a0014 processed
169/265: 184401 ddd:010082929:mpeg21:a0020 processed
170/265: 184401 ddd:010082929:mpeg21:a0021 processed
171/265: 184401 ddd:010082929:mpeg21:a0022 processed
172/265: 184401 ddd:010082929:mpeg21:a0024 processed
173/265: 184401 ddd:010082930:mpeg21:a0001 processed
174/265: 184401 ddd:010082930:mpeg21:a0002 processed
175/265: 184401 ddd:010082930:mpeg21:a0003 processed
176/265: 184401 ddd:010082930:mpeg21:a0004 processed
177/265: 184401 ddd:010082930:mpeg21:a0005 processed
178/265: 184401 ddd:010082930:mpeg21:a0006 processed
179/265: 184401 ddd:010082930:mpeg21:a0007 processed
180/265: 184401 ddd:010082930:mpeg21:a0012 processed
181/265: 184401 ddd:010082930:mpeg21:a0013 pro

66/254: 184402 ddd:010082938:mpeg21:a0020 processed
67/254: 184402 ddd:010082938:mpeg21:a0021 processed
68/254: 184402 ddd:010082939:mpeg21:a0001 processed
69/254: 184402 ddd:010082939:mpeg21:a0002 processed
70/254: 184402 ddd:010082939:mpeg21:a0008 processed
71/254: 184402 ddd:010082939:mpeg21:a0009 processed
72/254: 184402 ddd:010082939:mpeg21:a0012 processed
73/254: 184402 ddd:010082939:mpeg21:a0013 processed
74/254: 184402 ddd:010082939:mpeg21:a0014 processed
75/254: 184402 ddd:010082939:mpeg21:a0015 processed
76/254: 184402 ddd:010082939:mpeg21:a0028 processed
77/254: 184402 ddd:010082939:mpeg21:a0029 processed
78/254: 184402 ddd:010082939:mpeg21:a0030 processed
79/254: 184402 ddd:010082939:mpeg21:a0034 processed
80/254: 184402 ddd:010082939:mpeg21:a0036 processed
81/254: 184402 ddd:010082939:mpeg21:a0037 processed
82/254: 184402 ddd:010082939:mpeg21:a0038 processed
83/254: 184402 ddd:010082939:mpeg21:a0040 processed
84/254: 184402 ddd:010082939:mpeg21:a0041 processed
85/254: 1844

225/254: 184402 ddd:010082945:mpeg21:a0004 processed
226/254: 184402 ddd:010082945:mpeg21:a0005 processed
227/254: 184402 ddd:010082945:mpeg21:a0012 processed
228/254: 184402 ddd:010082945:mpeg21:a0013 processed
229/254: 184402 ddd:010082945:mpeg21:a0014 processed
230/254: 184402 ddd:010082945:mpeg21:a0015 processed
231/254: 184402 ddd:010082945:mpeg21:a0016 processed
232/254: 184402 ddd:010082945:mpeg21:a0027 processed
233/254: 184402 ddd:010082945:mpeg21:a0028 processed
234/254: 184402 ddd:010082945:mpeg21:a0029 processed
235/254: 184402 ddd:010082945:mpeg21:a0030 processed
236/254: 184402 ddd:010082945:mpeg21:a0032 processed
237/254: 184402 ddd:010082945:mpeg21:a0034 processed
238/254: 184402 ddd:010082946:mpeg21:a0017 processed
239/254: 184402 ddd:010082946:mpeg21:a0004 processed
240/254: 184402 ddd:010082946:mpeg21:a0005 processed
241/254: 184402 ddd:010082946:mpeg21:a0009 processed
242/254: 184402 ddd:010082946:mpeg21:a0010 processed
243/254: 184402 ddd:010082946:mpeg21:a0014 pro

140/230: 184403 ddd:010082953:mpeg21:a0029 processed
141/230: 184403 ddd:010082954:mpeg21:a0001 processed
142/230: 184403 ddd:010082954:mpeg21:a0002 processed
143/230: 184403 ddd:010082954:mpeg21:a0004 processed
144/230: 184403 ddd:010082954:mpeg21:a0005 processed
145/230: 184403 ddd:010082954:mpeg21:a0009 processed
146/230: 184403 ddd:010082954:mpeg21:a0012 processed
147/230: 184403 ddd:010082954:mpeg21:a0013 processed
148/230: 184403 ddd:010082954:mpeg21:a0014 processed
149/230: 184403 ddd:010082954:mpeg21:a0023 processed
150/230: 184403 ddd:010082954:mpeg21:a0025 processed
151/230: 184403 ddd:010082954:mpeg21:a0026 processed
152/230: 184403 ddd:010082954:mpeg21:a0027 processed
153/230: 184403 ddd:010082954:mpeg21:a0003 processed
154/230: 184403 ddd:010082954:mpeg21:a0006 processed
155/230: 184403 ddd:010082954:mpeg21:a0007 processed
156/230: 184403 ddd:010082954:mpeg21:a0008 processed
157/230: 184403 ddd:010082954:mpeg21:a0011 processed
158/230: 184403 ddd:010082954:mpeg21:a0029 pro

...

287/628: 184412 ddd:010083118:mpeg21:a0014 processed
288/628: 184412 ddd:010083118:mpeg21:a0015 processed
289/628: 184412 ddd:010083118:mpeg21:a0017 processed
290/628: 184412 ddd:010083118:mpeg21:a0018 processed
291/628: 184412 ddd:010083118:mpeg21:a0019 processed
292/628: 184412 ddd:010083118:mpeg21:a0020 processed
293/628: 184412 ddd:010083118:mpeg21:a0021 processed
294/628: 184412 ddd:010083118:mpeg21:a0023 processed
295/628: 184412 ddd:010083118:mpeg21:a0001 processed
296/628: 184412 ddd:010083118:mpeg21:a0002 processed
297/628: 184412 ddd:010083118:mpeg21:a0003 processed
298/628: 184412 ddd:010083118:mpeg21:a0004 processed
299/628: 184412 ddd:010083118:mpeg21:a0005 processed
300/628: 184412 ddd:010083119:mpeg21:a0001 processed
301/628: 184412 ddd:010083119:mpeg21:a0002 processed
302/628: 184412 ddd:010083119:mpeg21:a0003 processed
303/628: 184412 ddd:010083119:mpeg21:a0004 processed
304/628: 184412 ddd:010083119:mpeg21:a0005 processed
305/628: 184412 ddd:010083119:mpeg21:a0006 pro

449/628: 184412 ddd:010083126:mpeg21:a0018 processed
450/628: 184412 ddd:010083126:mpeg21:a0021 processed
451/628: 184412 ddd:010083126:mpeg21:a0022 processed
452/628: 184412 ddd:010083126:mpeg21:a0023 processed
453/628: 184412 ddd:010083126:mpeg21:a0002 processed
454/628: 184412 ddd:010083126:mpeg21:a0003 processed
455/628: 184412 ddd:010083126:mpeg21:a0004 processed
456/628: 184412 ddd:010083126:mpeg21:a0001 processed
457/628: 184412 ddd:010083126:mpeg21:a0007 processed
458/628: 184412 ddd:010083126:mpeg21:a0011 processed
459/628: 184412 ddd:010083126:mpeg21:a0012 processed
460/628: 184412 ddd:010083126:mpeg21:a0019 processed
461/628: 184412 ddd:010083127:mpeg21:a0001 processed
462/628: 184412 ddd:010083127:mpeg21:a0002 processed
463/628: 184412 ddd:010083127:mpeg21:a0003 processed
464/628: 184412 ddd:010083127:mpeg21:a0004 processed
465/628: 184412 ddd:010083127:mpeg21:a0005 processed
466/628: 184412 ddd:010083127:mpeg21:a0006 processed
467/628: 184412 ddd:010083127:mpeg21:a0007 pro

608/628: 184412 ddd:010083135:mpeg21:a0032 processed
609/628: 184412 ddd:010083135:mpeg21:a0038 processed
610/628: 184412 ddd:010083135:mpeg21:a0039 processed
611/628: 184412 ddd:010083135:mpeg21:a0040 processed
612/628: 184412 ddd:010083135:mpeg21:a0041 processed
613/628: 184412 ddd:010083135:mpeg21:a0042 processed
614/628: 184412 ddd:010083135:mpeg21:a0043 processed
615/628: 184412 ddd:010083135:mpeg21:a0044 processed
616/628: 184412 ddd:010083135:mpeg21:a0045 processed
617/628: 184412 ddd:010083135:mpeg21:a0047 processed
618/628: 184412 ddd:010083135:mpeg21:a0049 processed
619/628: 184412 ddd:010083135:mpeg21:a0050 processed
620/628: 184412 ddd:010083135:mpeg21:a0051 processed
621/628: 184412 ddd:010083135:mpeg21:a0052 processed
622/628: 184412 ddd:010083135:mpeg21:a0056 processed
623/628: 184412 ddd:010083135:mpeg21:a0057 processed
624/628: 184412 ddd:010083135:mpeg21:a0058 processed
625/628: 184412 ddd:010083135:mpeg21:a0059 processed
626/628: 184412 ddd:010083135:mpeg21:a0060 pro

140/642: 184501 ddd:010082572:mpeg21:a0031 processed
141/642: 184501 ddd:010082572:mpeg21:a0033 processed
142/642: 184501 ddd:010082572:mpeg21:a0034 processed
143/642: 184501 ddd:010082572:mpeg21:a0036 processed
144/642: 184501 ddd:010082572:mpeg21:a0037 processed
145/642: 184501 ddd:010082572:mpeg21:a0038 processed
146/642: 184501 ddd:010082572:mpeg21:a0040 processed
147/642: 184501 ddd:010082572:mpeg21:a0049 processed
148/642: 184501 ddd:010082572:mpeg21:a0051 processed
149/642: 184501 ddd:010082572:mpeg21:a0052 processed
150/642: 184501 ddd:010082574:mpeg21:a0001 processed
151/642: 184501 ddd:010082574:mpeg21:a0002 processed
152/642: 184501 ddd:010082574:mpeg21:a0003 processed
153/642: 184501 ddd:010082574:mpeg21:a0004 processed
154/642: 184501 ddd:010082574:mpeg21:a0005 processed
155/642: 184501 ddd:010082574:mpeg21:a0006 processed
156/642: 184501 ddd:010082574:mpeg21:a0007 processed
157/642: 184501 ddd:010082574:mpeg21:a0008 processed
158/642: 184501 ddd:010082574:mpeg21:a0010 pro

KeyboardInterrupt: 